In [2]:
import os
import json
import random
import pandas as pd
from pathlib import Path
from rich.console import Console
from rich.panel import Panel
from rich import inspect
from rich.text import Text
from rich.progress import Progress
import json

console = Console()

folder_aws_path = "aws/"
folder_build_path = "build/"
folder_build_config_path = "build/config/"
folder_build_task_path = "build/task/"
folder_build_mturk_path = "build/mturk/"
folder_build_deploy_path = "build/deploy/"
folder_build_skeleton_path = "build/skeleton/"
folder_tasks_path = "tasks/"

def serialize_json(folder, filename, data):
    if not os.path.exists(folder):
        os.makedirs(folder, exist_ok=True)
    with open(f"{folder}/{filename}", 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4, default=str)
        f.close()

def remove_json(folder, filename):
    os.remove(f"{folder}/{filename}")

def read_json(path):
    if os.path.exists(path):
        with open(path, "r", encoding="utf8") as file:
            data = json.load(file)
        return data
    else:
        return {}

## Section 1 - Environment variables loading

Remember to restart Jupyter if you edit environment variables values

In [3]:
from dotenv import load_dotenv
from distutils.util import strtobool

env_path = Path('.') / '.env'
load_dotenv(dotenv_path=env_path)

task_name = os.getenv('task_name')
batch_name = os.getenv('batch_name')
admin_user = os.getenv('admin_user')
admin_password = os.getenv('admin_password')
deploy_config = strtobool(os.getenv('deploy_config'))

aws_region = os.getenv('aws_region')
aws_private_bucket = os.getenv('aws_private_bucket')
aws_deploy_bucket = os.getenv('aws_deploy_bucket')

bing_api_key = os.getenv('bing_api_key')


## Section 2 - Setting up IAM policies and identity
### build/aws

In [4]:
import boto3
from botocore.exceptions import ClientError

iam = boto3.client('iam')
iam_resource = boto3.resource('iam')

console.rule("Root user identity")

root_user = iam_resource.CurrentUser()
aws_account_id = root_user.arn.split(':')[4]

console.print(f"ID: [bold cyan on white]{root_user.user_id}")
console.print(f"Username: [bold cyan on white]{root_user.user_name}")
console.print(f"ARN: [bold cyan on white]{root_user.arn}")
console.print(f"AWS Account ID: [bold cyan on white]{aws_account_id}")

console.rule("IAM policy [cyan underline]crowd-workers-dev[/cyan underline]")

crowd_workers_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "allowBucketInteraction",
            "Effect": "Allow",
            "Action": [
                "s3:PutObject",
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                f"arn:aws:s3:::{aws_private_bucket}",
                f"arn:aws:s3:::{aws_private_bucket}/*",
                f"arn:aws:s3:::{aws_deploy_bucket}",
                f"arn:aws:s3:::{aws_deploy_bucket}/*"
            ]
        },
        {
            "Sid": "allowDatabaseInteraction",
            "Effect": "Allow",
            "Action": [
                "dynamodb:DescribeTable",
                "dynamodb:DeleteItem",
                "dynamodb:PutItem",
                "dynamodb:GetItem",
            ],
            "Resource": f"arn:aws:dynamodb:{aws_region}:{aws_account_id}:table/users"
        }
    ]
}


{
    "Version": "2012-10-17",
    "Statement": [

    ]
}

policy = None
try:
    policy = iam.create_policy(
        PolicyName='crowd-workers-dev',
        PolicyDocument=json.dumps(crowd_workers_policy)
    )
    console.print(
        f"[green]Policy creation completed[/green], HTTP STATUS CODE: {policy['ResponseMetadata']['HTTPStatusCode']}.")
except (iam.exceptions.EntityAlreadyExistsException) as exception:
    console.print(f"[yellow]Policy already present[/yellow]")
    policy = iam.get_policy(PolicyArn=f"arn:aws:iam::{aws_account_id}:policy/crowd-workers-dev")
    console.print(
        f"[green]Policy retrieved[/green], HTTP STATUS CODE: {policy['ResponseMetadata']['HTTPStatusCode']}.")
serialize_json(folder_aws_path, f"policy_{policy['Policy']['PolicyName']}.json", policy)

console.print(f"Policy ARN: [cyan underline]{policy['Policy']['Arn']}[/cyan underline]")

console.rule("IAM user [cyan underline]worker-dev[/cyan underline]")

user = None
try:
    user = iam.create_user(UserName="worker-dev")
    console.print(
        f"[green]user created[/green], HTTP STATUS CODE: {user['ResponseMetadata']['HTTPStatusCode']}.")
except (iam.exceptions.EntityAlreadyExistsException) as exception:
    console.print(f"[yellow]User already present[/yellow]")
    user = iam.get_user(UserName="worker-dev")
    console.print(
        f"[green]User retrieved[green], HTTP STATUS CODE: {user['ResponseMetadata']['HTTPStatusCode']}.")
serialize_json(folder_aws_path, f"user_{user['User']['UserName']}_data.json", user)

response = iam.attach_user_policy(UserName=user['User']['UserName'], PolicyArn=policy['Policy']['Arn'])
policy = iam.get_policy(PolicyArn=f"{policy['Policy']['Arn']}")
console.print(
    f"[green]Policy with ARN [cyan underline]{policy['Policy']['Arn']}[/cyan underline] attached to user, HTTP STATUS CODE: {user['ResponseMetadata']['HTTPStatusCode']}")

keys = []
paginator = iam.get_paginator('list_access_keys')
for found_keys in paginator.paginate(UserName=user['User']['UserName']):
    for (index, key) in enumerate(found_keys['AccessKeyMetadata']):
        keyData = read_json(f"{folder_aws_path}user_{user['User']['UserName']}_access_key_{key['AccessKeyId']}.json")
        if keyData:
            keys.append(keyData)
        else:
            response = iam.delete_access_key(UserName=user['User']['UserName'], AccessKeyId=key['AccessKeyId'])
            console.print(f"[red]Key {index} data not found on disk[/red]; deleting it on AWS, HTTP STATUS CODE: {response['ResponseMetadata']['HTTPStatusCode']}")

if len(keys) < 2:
    key = iam.create_access_key(UserName=user['User']['UserName'])
    serialize_json(folder_aws_path, f"user_{user['User']['UserName']}_access_key_{key['AccessKey']['AccessKeyId']}.json", key)
    console.print(f"[green]Access key created[/green], HTTP STATUS CODE: {key['ResponseMetadata']['HTTPStatusCode']}.")
    keys.append(key)
    if not os.path.exists(f"{folder_aws_path}user_{user['User']['UserName']}_access_key_{key['AccessKey']['AccessKeyId']}.json"):
        serialize_json(folder_aws_path, f"user_{user['User']['UserName']}_access_key_{key['AccessKey']['AccessKeyId']}.json", key)
        console.print(f"[green]Access key created[/green], HTTP STATUS CODE: {key['ResponseMetadata']['HTTPStatusCode']}.")

key_selected = random.choice(keys)
key_data = read_json(f"{folder_aws_path}user_{user['User']['UserName']}_access_key_{key_selected['AccessKey']['AccessKeyId']}.json")

console.print("Key data found on disk and loaded")

aws_worker_access_id = key_data['AccessKey']['AccessKeyId']
aws_worker_access_secret = key_data['AccessKey']['SecretAccessKey']


──────────────────────────────────── Root user identity ─────────────────────────────────────

ID: AIDAT5QYFVEAWTOZD6JBO

Username: dev_sdk

ARN: arn:aws:iam::269559900417:user/dev_sdk

AWS Account ID: 269559900417

...

──────────────────────────────────── IAM user worker-dev ────────────────────────────────────

User already present

User retrieved, HTTP STATUS CODE: 200.

Policy with ARN arn:aws:iam::269559900417:policy/crowd-workers-dev attached to user, HTTP 
STATUS CODE: 200

Key data found on disk and loaded

## Section 3 - Private and deploy bucket creation
### build/aws

In [5]:
s3_client = boto3.client('s3')
s3_resource = boto3.resource('s3')

buckets = []
for bucket in s3_resource.buckets.all():
    buckets.append(bucket.name)

console.rule(f"bucket [cyan underline]{aws_private_bucket}[/cyan underline]")

try:
    private_bucket = s3_client.create_bucket(
        Bucket=aws_private_bucket,
        CreateBucketConfiguration={
            'LocationConstraint': aws_region
        }
    )
    console.print(f"[green]Bucket creation completed[/green], HTTP STATUS CODE: {private_bucket['ResponseMetadata']['HTTPStatusCode']}.")
except s3_client.exceptions.BucketAlreadyOwnedByYou as error:
    private_bucket = s3_resource.Bucket(aws_private_bucket)
    console.print(f"[yellow]Bucket already present[/yellow], HTTP STATUS CODE: {error.response['ResponseMetadata']['HTTPStatusCode']}.")
serialize_json(folder_aws_path, f"bucket_{aws_private_bucket}.json", private_bucket)

response = s3_client.put_public_access_block(
    Bucket=aws_private_bucket,
    PublicAccessBlockConfiguration={
        'BlockPublicAcls': True,
        'IgnorePublicAcls': True,
        'BlockPublicPolicy': True,
        'RestrictPublicBuckets': True
    },
)
console.print(f"[green]Public access blocked[/green], HTTP STATUS CODE: {response['ResponseMetadata']['HTTPStatusCode']}.")

private_bucket_policy = {
    "Version": "2012-10-17",
    "Id": "private-bucket-policy",
    "Statement": [
        {
            "Sid": "allow-bucket-interaction",
            "Effect": "Allow",
            "Principal": {
                "AWS": f"arn:aws:iam::{aws_account_id}:user/{user['User']['UserName']}",
            },
            "Action": [
                "s3:PutObject",
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                f"arn:aws:s3:::{aws_private_bucket}",
                f"arn:aws:s3:::{aws_private_bucket}/*"
            ]
        }
    ]
}

try:
    policy = s3_client.get_bucket_policy(Bucket=aws_private_bucket)
    policy['Policy'] = json.loads(policy['Policy'])
    console.print(f"[yellow]Policy already present[/yellow], HTTP STATUS CODE: {response['ResponseMetadata']['HTTPStatusCode']}.")
except ClientError as e:
    if e.response['Error']['Code'] == 'NoSuchBucketPolicy':
        response = s3_client.put_bucket_policy(Bucket=aws_private_bucket, Policy=json.dumps(private_bucket_policy))
        console.print(f"[green]Policy configuration completed[/green], HTTP STATUS CODE: {response['ResponseMetadata']['HTTPStatusCode']}.")
    policy = s3_client.get_bucket_policy(Bucket=aws_private_bucket)
    policy['Policy'] = json.loads(policy['Policy'])
serialize_json(folder_aws_path, f"bucket_{aws_private_bucket}_policy.json", policy)

cors_configuration = {
    'CORSRules': [{
        'AllowedHeaders': ['*'],
        'AllowedMethods': ['GET', 'HEAD', 'PUT'],
        'AllowedOrigins': ['*'],
        'ExposeHeaders': [],
        'MaxAgeSeconds': 3000
    }]
}

try:
    cors_configuration = s3_client.get_bucket_cors(Bucket=aws_private_bucket)
    console.print(f"[yellow]CORS Configuration already present[/yellow], HTTP STATUS CODE: {response['ResponseMetadata']['HTTPStatusCode']}.")
except ClientError as e:
    if e.response['Error']['Code'] == 'NoSuchCORSConfiguration':
        response = s3_client.put_bucket_cors(Bucket=aws_private_bucket, CORSConfiguration=cors_configuration)
        console.print(f"[green]CORS configuration completed[green], HTTP STATUS CODE: {response['ResponseMetadata']['HTTPStatusCode']}.")
cors_configuration = s3_client.get_bucket_cors(Bucket=aws_private_bucket)
serialize_json(folder_aws_path, f"bucket_{aws_private_bucket}_cors.json", cors_configuration)

console.rule(f"bucket [cyan underline]{aws_deploy_bucket}[/cyan underline]")

try:
    deploy_bucket = s3_client.create_bucket(
        Bucket=aws_deploy_bucket,
        CreateBucketConfiguration={
            'LocationConstraint': aws_region
        }
    )
    console.print(f"[green]Bucket creation completed[/green], HTTP STATUS CODE: {deploy_bucket['ResponseMetadata']['HTTPStatusCode']}.")
except s3_client.exceptions.BucketAlreadyOwnedByYou as error:
    deploy_bucket = s3_resource.Bucket(aws_deploy_bucket)
    console.print(f"[yellow]Bucket already present[/yellow], HTTP STATUS CODE: {error.response['ResponseMetadata']['HTTPStatusCode']}.")
serialize_json(folder_aws_path, f"bucket_{aws_deploy_bucket}.json", deploy_bucket)

deploy_bucket_policy = {
    "Version": "2012-10-17",
    "Id": "deploy-bucket-policy",
    "Statement": [
        {
            "Sid": "allow-bucket-interaction",
            "Effect": "Allow",
            "Principal": {
                "AWS": f"arn:aws:iam::{aws_account_id}:user/{user['User']['UserName']}"
            },
            "Action": [
                "s3:PutObject",
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                f"arn:aws:s3:::{aws_deploy_bucket}",
                f"arn:aws:s3:::{aws_deploy_bucket}/*"
            ]
        },
        {
            "Sid": "allow-bucket-administration",
            "Effect": "Allow",
            "Principal": {
                "AWS": root_user.arn
            },
            "Action": [
                "s3:*",
            ],
            "Resource": [
                f"arn:aws:s3:::{aws_deploy_bucket}",
                f"arn:aws:s3:::{aws_deploy_bucket}/*"
            ]
        }
    ]
}

try:
    policy = s3_client.get_bucket_policy(Bucket=aws_deploy_bucket)
    policy['Policy'] = json.loads(policy['Policy'])
    console.print(f"[yellow]Policy already present[/yellow], HTTP STATUS CODE: {response['ResponseMetadata']['HTTPStatusCode']}.")
except ClientError as e:
    if e.response['Error']['Code'] == 'NoSuchBucketPolicy':
        response = s3_client.put_bucket_policy(Bucket=aws_deploy_bucket, Policy=json.dumps(deploy_bucket_policy))
        console.print(f"[green]Policy configuration completed[/green], HTTP STATUS CODE: {response['ResponseMetadata']['HTTPStatusCode']}.")
    policy = s3_client.get_bucket_policy(Bucket=aws_deploy_bucket)
    policy['Policy'] = json.loads(policy['Policy'])
serialize_json(folder_aws_path, f"bucket_{aws_private_bucket}_policy.json", policy)


─────────────────────────────── bucket crowdsourcing-tasks-us ───────────────────────────────

Bucket already present, HTTP STATUS CODE: 409.

Public access blocked, HTTP STATUS CODE: 200.

Policy already present, HTTP STATUS CODE: 200.

CORS Configuration already present, HTTP STATUS CODE: 200.

────────────────────────────── bucket crowdsourcing-deploy-us ───────────────────────────────

Bucket already present, HTTP STATUS CODE: 409.

Policy already present, HTTP STATUS CODE: 200.

# Section 4 - Environment file generation
### build/environments

In [6]:
environment_development = f"{folder_build_path}environments/environment.ts"
environment_production = f"{folder_build_path}environments/environment.prod.ts"

console.rule("Environment: [cyan underline]PRODUCTION[/cyan underline]")

environment_dict = {
    "production": 'true',
    "configuration_local": 'false',
    "taskName": task_name,
    "batchName": batch_name,
    "region": aws_region,
    "bucket": aws_private_bucket,
    "aws_id_key": aws_worker_access_id,
    "aws_secret_key": aws_worker_access_secret,
    "bing_api_key": bing_api_key
}

with open(environment_production, 'w') as file:
    print("export const environment = {", file=file)
    for (env_var, value) in environment_dict.items():
        if env_var == 'production' or env_var == 'configuration_local':
            print(f"\t{env_var}: {value},", file=file)
        else:
            print(f"\t{env_var}: \"{value}\",", file=file)
    print("};", file=file)

console.print("File [cyan underline]environment.prod.ts[/cyan underline] generated")
console.print(f"Path: [italic]{environment_production}[/italic]")

console.rule("Environment: [cyan underline]DEVELOPMENT[/cyan underline]")

environment_dict = {
    "production": 'false',
    "configuration_local": 'true',
    "taskName": task_name,
    "batchName": batch_name,
    "region": aws_region,
    "bucket": aws_private_bucket,
    "aws_id_key": aws_worker_access_id,
    "aws_secret_key": aws_worker_access_secret,
    "bing_api_key": bing_api_key
}

with open(environment_development, 'w') as file:
    print("export const environment = {", file=file)
    for (env_var, value) in environment_dict.items():
        if env_var == 'production' or env_var == 'configuration_local':
            print(f"\t{env_var}: {value},", file=file)
        else:
            print(f"\t{env_var}: \"{value}\",", file=file)
    print("};", file=file)

console.print("File [cyan underline]environment.ts[/cyan underline] generated")
console.print(f"Path: [italic]{environment_development}[/italic]")

────────────────────────────────── Environment: PRODUCTION ──────────────────────────────────

File environment.prod.ts generated

Path: build/environments/environment.prod.ts

───────────────────────────────── Environment: DEVELOPMENT ──────────────────────────────────

File environment.ts generated

Path: build/environments/environment.ts

## Section 5 - admin.json file generation
### build/config

In [7]:
import hmac
import hashlib

console.rule("File [cyan underline]admin.json")

if not os.path.exists(folder_build_config_path):
    os.makedirs(folder_build_config_path, exist_ok=True)

admin_file = f"{folder_build_config_path}admin.json"

console.print("Creating hash with [cyan underline]hmac[/cyan underline] and [cyan underline]sha256[/cyan underline]")
console.print(f"Processing user with username: [white on purple]{admin_user}[white on purple]")

admins = []
body = f"username:{admin_user}"
digest_maker = hmac.new(admin_password.encode(), body.encode(), hashlib.sha256)
admins.append(digest_maker.hexdigest())
with open(admin_file, 'w') as file:
    json.dump(admins, file, indent=4)

console.print(f"Path: [italic]{admin_file}")

────────────────────────────────────── File admin.json ──────────────────────────────────────

Creating hash with hmac and sha256

Processing user with username: admin

Path: build/config/admin.json

## Section 6 - document.ts file generation
### build/skeleton

In [8]:
import textwrap

console.rule("Interface [cyan underline]document.ts")

hits_file = f"{folder_build_task_path}hits.json"
document_interface = f"{folder_build_skeleton_path}document.ts"
if not os.path.exists(folder_build_skeleton_path):
    os.makedirs(folder_build_skeleton_path, exist_ok=True)

console.print(f"Reading hits file")
console.print(f"Path: [italic]{hits_file}[/italic]")
hits = read_json(hits_file)
sample_element = hits.pop()['documents'].pop()

if not 'id' in sample_element.keys():
    raise Exception("[red]Your [underline]hits.json[/underline] file contains an attributed called [underline]\"id\"[/underline]?")

# This class provides a representation of a single document stored in single hit stored in the Amazon S3 bucket.
# The attribute <document_index> is additional and should not be touched and passed in the constructor.
# Each field of such Document must be mapped to an attribute of this class and set up in the constructor as it is shown.

with open(document_interface, 'w') as file:
    print("export class Document {", file=file)
    print("", file=file)
    wrapper = textwrap.TextWrapper(initial_indent='\t\t', subsequent_indent='\t\t')
    print(wrapper.fill("index: number;"), file=file)
    print(wrapper.fill("countdownExpired: boolean;"), file=file)
    for attribute, value in sample_element.items():
        try:
            element = json.loads(value)
            if isinstance(element, dict):
                print(wrapper.fill(f"{attribute}: Array<JSON>;"), file=file)
            elif isinstance(element, int) or isinstance(element, float):
                if(attribute=="id"):
                    print(wrapper.fill(f"{attribute}: string;"), file=file)
                else:
                    print(wrapper.fill(f"{attribute}: number;"), file=file)
            elif isinstance(element, list):
                print(wrapper.fill(f"{attribute}: Array<String>;"), file=file)
            else:
                print(wrapper.fill(f"{attribute}: string;"), file=file)
            console.print(f"Attribute with name: [cyan underline]{attribute}[/cyan underline] and type: {type(element)} found")
        except (TypeError, ValueError) as e:
            if isinstance(value, list):
                print(wrapper.fill(f"{attribute}: Array<String>;"), file=file)
            elif isinstance(value, int) or isinstance(value, float):
                print(wrapper.fill(f"{attribute}: number;"), file=file)
            else:
                print(wrapper.fill(f"{attribute}: string;"), file=file)
            console.print(f"Attribute with name: [cyan underline]{attribute}[/cyan underline] and type: {type(value)} found")
    print("", file=file)
    print(wrapper.fill(f"constructor ("), file=file)
    wrapper = textwrap.TextWrapper(initial_indent='\t\t\t', subsequent_indent='\t\t\t')
    print(wrapper.fill("index: number,"), file=file)
    print(wrapper.fill("data: JSON"), file=file)
    wrapper = textwrap.TextWrapper(initial_indent='\t\t', subsequent_indent='\t\t')
    print(wrapper.fill(") {"), file=file)
    print("", file=file)
    wrapper = textwrap.TextWrapper(initial_indent='\t\t\t', subsequent_indent='\t\t\t')
    print(wrapper.fill("this.index = index"), file=file)
    for attribute, value in sample_element.items():
        try:
            element = json.loads(value)
            if isinstance(element, dict):
                print(wrapper.fill(f"this.{attribute} = new Array<JSON>()"), file=file)
                print(wrapper.fill(f"for (let index = 0; index < data[\"{attribute}\"].length; index++) this.{attribute}.push(data[\"{attribute}\"][index])"), file=file)
            elif isinstance(element, list):
                print(wrapper.fill(f"this.{attribute} = new Array<String>()"), file=file)
                print(wrapper.fill(f"for (let index = 0; index < data[\"{attribute}\"].length; index++) this.{attribute}.push(data[\"{attribute}\"])"), file=file)
            else:
                wrapper = textwrap.TextWrapper(initial_indent='\t\t\t', subsequent_indent='\t\t\t')
                print(wrapper.fill(f"this.{attribute} = data[\"{attribute}\"]"), file=file)
        except (TypeError, ValueError) as e:
            if isinstance(value, list):
                print(wrapper.fill(f"this.{attribute} = new Array<String>()"), file=file)
                print(wrapper.fill(f"for (let index = 0; index < data[\"{attribute}\"].length; index++) this.{attribute}.push(data[\"{attribute}\"])"), file=file)
            else:
                wrapper = textwrap.TextWrapper(initial_indent='\t\t\t', subsequent_indent='\t\t\t')
                print(wrapper.fill(f"this.{attribute} = data[\"{attribute}\"]"), file=file)
    wrapper = textwrap.TextWrapper(initial_indent='\t\t', subsequent_indent='\t\t')
    print("", file=file)
    print(wrapper.fill("}"), file=file)
    print("", file=file)
    print("}", file=file)

console.print("Interface built")
console.print(f"Path: [italic]{document_interface}[/italic]")

─────────────────────────────────── Interface document.ts ───────────────────────────────────

Reading hits file

Path: build/task/hits.json

Attribute with name: name_unique and type: <class 'str'> found

Attribute with name: statement and type: <class 'str'> found

...

Attribute with name: party and type: <class 'str'> found

Attribute with name: source and type: <class 'str'> found

Attribute with name: id and type: <class 'int'> found

Interface built

Path: build/skeleton/document.ts

## Section 7 - Amazon Mechanical Turk assets generation
### build/mturk

In [9]:
from mako.template import Template

console.rule("Amazon Mechanical Turk landing page")

model = Template(filename=f"{folder_build_mturk_path}model.html")
mturk_page = model.render(
    aws_region = aws_region,
    aws_deploy_bucket = aws_deploy_bucket,
    task_name = task_name,
    batch_name = batch_name
)
mturk_page_file = f"{folder_build_mturk_path}index.html"
with open(mturk_page_file, 'w') as file:
    print(mturk_page, file=file)

console.print(f"Model istantiated")
console.print(f"Path: {mturk_page_file}")

hits_file = f"{folder_build_task_path}hits.json"
mturk_tokens_file = f"{folder_build_mturk_path}tokens.csv"
console.print(f"Loading [cyan underline]hits.json[/cyan underline] file")
console.print(f"Path: [ital]{hits_file}")
hits = read_json(hits_file)
token_df = pd.DataFrame(columns=["token_input", "token_output"])
for hit in hits:
    token_df = token_df.append({
        "token_input": hit['token_input'],
        "token_output": hit['token_output']
    }, ignore_index=True)
token_df.to_csv(mturk_tokens_file, index=False)
console.print(f"Tokens for {len(hits)} generated")
console.print(f"Path: [italic]{mturk_tokens_file}")

──────────────────────────── Amazon Mechanical Turk landing page ────────────────────────────

Model istantiated

Path: build/mturk/index.html

Loading hits.json file

Path: build/task/hits.json

Tokens for 200 generated

Path: build/mturk/tokens.csv

## Section 8 - Angular Application Build
### build/deploy

In [10]:
import subprocess
import shutil
from mako.template import Template
import time

folder_build_result = f"../dist/"

console.rule(f"Task [cyan underline]{task_name}[/cyan underline]/[yellow underline]{batch_name}[/yellow underline] build")

console.print("Executing command")
command = "ng build --configuration=\"production\" --output-hashing=none"
console.print(f"[green on black]{command}")
console.print(f"Please wait...")
process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE)
for line in process.stdout:
    line_clean = line.decode().strip()
    if "Initial Total" in line_clean:
        line_clean = line_clean[2:]
    if line_clean!="":
        console.print(line_clean)
process.wait()

console.print("Merging Javascript assets")
script_merged_file = f"{folder_build_deploy_path}scripts.js"
if(os.path.exists(script_merged_file)):
    os.remove(script_merged_file)
es_scripts = [
    'polyfills.js',
    'runtime.js',
    'main.js',
]
with open(script_merged_file, 'a') as outfile:
    for file in es_scripts:
        script_current_file = f"{folder_build_result}Crowd_Frame/{file}"
        console.print(f"Processing file: [italic purple on black]{script_current_file}")
        with open(script_current_file) as script:
            for line in script:
                outfile.write(line)
console.print(f"Path: [italic]{script_merged_file}")

console.print("Merging CSS assets")
styles_merged_file = f"{folder_build_deploy_path}styles.css"
if(os.path.exists(styles_merged_file)):
    os.remove(styles_merged_file)
css_styles = ['styles.css']
with open(styles_merged_file, 'a') as outfile:
    for file in css_styles:
        style_current_file = f"{folder_build_result}Crowd_Frame/{file}"
        console.print(f"Processing file: [italic cyan on black]{style_current_file}")
        with open(style_current_file) as style:
            for line in style:
                outfile.write(line)
console.print(f"Path: [italic underline]{styles_merged_file}")

console.print("Deleting build folder")
console.print(f"Path: [italic underline]{folder_build_result}")
shutil.rmtree(folder_build_result)

model = Template(filename=f"{folder_build_deploy_path}model.html")
index_page = model.render(
    task_name = task_name,
    batch_name = batch_name
)
index_page_file = f"{folder_build_deploy_path}index.html"
with open(index_page_file, 'w') as file:
    print(index_page, file=file)

console.print("Model istantiated")
console.print(f"Path: [italic underline]{index_page_file}")

────────────────────────────────── Task Test/Batch-1 build ──────────────────────────────────

Executing command

ng build --configuration="production" --output-hashing=none

Please wait...

Initial Chunk Files | Names         |      Size

...

Path: build/deploy/styles.css

Deleting build folder

Path: ../dist/

Model istantiated

Path: build/deploy/index.html

## Section 9 - Packaging
### tasks/task_name/batch_name

In [11]:
from shutil import copy2

console.rule(f"Packaging task in [cyan underline]tasks/{task_name}/{batch_name}")

folder_tasks_batch_path = f"{folder_tasks_path}{task_name}/{batch_name}/"
folder_tasks_batch_deploy_path = f"{folder_tasks_batch_path}deploy/"
folder_tasks_batch_mturk_path = f"{folder_tasks_batch_path}mturk/"
folder_tasks_batch_task_path = f"{folder_tasks_batch_path}task/"
folder_tasks_batch_config_path = f"{folder_tasks_batch_path}config/"

console.print(f"[italic purple]deploy-config[/italic purple] variable: {bool(deploy_config)}")

if not os.path.exists(folder_tasks_batch_deploy_path):
    console.print("[green]Deploy folder created")
    os.makedirs(folder_tasks_batch_deploy_path, exist_ok=True)
else:
    console.print("[yellow]Deploy folder already present")
console.print(f"Path: [italic]{folder_tasks_batch_deploy_path}")
if not os.path.exists(folder_tasks_batch_mturk_path):
    console.print("[green]Amazon Mechanical Turk assets folder created")
    os.makedirs(folder_tasks_batch_mturk_path, exist_ok=True)
else:
    console.print("[yellow]Amazon Mechanical Turk assets folder already present")
console.print(f"Path: [italic]{folder_tasks_batch_mturk_path}")
if not os.path.exists(folder_tasks_batch_task_path) and deploy_config:
    console.print("[green]Task configuration folder created")
    os.makedirs(folder_tasks_batch_task_path, exist_ok=True)
else:
    console.print("[yellow]Task configuration folder already present")
console.print(f"Path: [italic]{folder_tasks_batch_task_path}")
if not os.path.exists(folder_tasks_batch_config_path) and deploy_config:
    console.print("[green]Task configuration folder created")
    os.makedirs(folder_tasks_batch_config_path, exist_ok=True)
else:
    console.print("[yellow]General configuration folder already present")
console.print(f"Path: [italic]{folder_tasks_batch_config_path}")

def copy(source, destination, title):
    panel = Panel(f"Source: [italic white on black]{source}[/italic white on black]\nDestination: [italic white on black]{destination}[/italic white on black]", title=title)
    console.print(panel)
    copy2(source, destination)

console.print(f"Copying files for [blue underline on white]{folder_build_deploy_path}[/blue underline on white] folder")

source = f"{folder_build_deploy_path}scripts.js"
destination = f"{folder_tasks_batch_deploy_path}scripts.js"
copy(source, destination, "Javascript Assets")

source = f"{folder_build_deploy_path}styles.css"
destination = f"{folder_tasks_batch_deploy_path}styles.css"
copy(source, destination, "CSS Styles")

source = f"{folder_build_deploy_path}index.html"
destination = f"{folder_tasks_batch_deploy_path}index.html"
copy(source, destination, "Task Homepage")

console.print(f"Copying files for [blue underline on white]{folder_build_mturk_path}[/blue underline on white] folder")

source = f"{folder_build_mturk_path}index.html"
destination = f"{folder_tasks_batch_mturk_path}index.html"
copy(source, destination, "Amazon Mechanical Turk landing page")

source = f"{folder_build_mturk_path}tokens.csv"
destination = f"{folder_tasks_batch_mturk_path}tokens.csv"
copy(source, destination, "Hits tokens")

if bool(deploy_config):

    console.print(f"Copying files for [blue underline on white]{folder_build_task_path}[/blue underline on white] folder")

    source = f"{folder_build_task_path}hits.json"
    destination = f"{folder_tasks_batch_task_path}hits.json"
    copy(source, destination, "Hits")

    source = f"{folder_build_task_path}dimensions.json"
    destination = f"{folder_tasks_batch_task_path}dimensions.json"
    copy(source, destination, "Dimensions")

    source = f"{folder_build_task_path}instructions_dimensions.json"
    destination = f"{folder_tasks_batch_task_path}instructions_dimensions.json"
    copy(source, destination, "Assessment Instructions")

    source = f"{folder_build_task_path}instructions_main.json"
    destination = f"{folder_tasks_batch_task_path}instructions_main.json"
    copy(source, destination, "General Instructions")

    source = f"{folder_build_task_path}questionnaires.json"
    destination = f"{folder_tasks_batch_task_path}questionnaires.json"
    copy(source, destination, "Questionnaires")

    source = f"{folder_build_task_path}search_engine.json"
    destination = f"{folder_tasks_batch_task_path}search_engine.json"
    copy(source, destination, "Search Engine")

    source = f"{folder_build_task_path}task.json"
    destination = f"{folder_tasks_batch_task_path}task.json"
    copy(source, destination, "Task Settings")

    source = f"{folder_build_task_path}workers.json"
    destination = f"{folder_tasks_batch_task_path}workers.json"
    copy(source, destination, "Workers Settings")

source = f"{folder_build_config_path}admin.json"
destination = f"{folder_tasks_batch_config_path}admin.json"
copy(source, destination, "Admin Credentials")

console.print(f"Copying files for [blue underline on white]{folder_tasks_batch_config_path}[/blue underline on white] folder")

─────────────────────────── Packaging task in tasks/Test/Batch-1 ────────────────────────────

deploy-config variable: True

Deploy folder already present

Path: tasks/Test/Batch-1/deploy/

Amazon Mechanical Turk assets folder already present

...

╭────────────────────────────────────── Search Engine ──────────────────────────────────────╮
│ Source: build/task/search_engine.json                                                     │
│ Destination: tasks/Test/Batch-1/task/search_engine.json                                   │
╰───────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────── Task Settings ──────────────────────────────────────╮
│ Source: build/task/task.json                                                              │
│ Destination: tasks/Test/Batch-1/task/task.json                                            │
╰───────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────── Workers Settings ─────────────────────────────────────╮
│ Source: build/task/workers.json                                                           │
│ Destination: tasks/Test/Batch-1/task/workers.json                                         │
╰───────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────── Admin Credentials ────────────────────────────────────╮
│ Source: build/config/admin.json                                                           │
│ Destination: tasks/Test/Batch-1/config/admin.json                                         │
╰───────────────────────────────────────────────────────────────────────────────────────────╯

Copying files for tasks/Test/Batch-1/config/ folder

## Section 10 - Deploy

In [12]:

s3_private_generator_path = f"{task_name}/{batch_name}/Generator/"
s3_private_task_path = f"{task_name}/{batch_name}/Task/"
s3_deploy_path = f"{task_name}/{batch_name}/"

folder_tasks_batch_deploy_path = f"{folder_tasks_batch_path}deploy/"
folder_tasks_batch_mturk_path = f"{folder_tasks_batch_path}mturk/"
folder_tasks_batch_task_path = f"{folder_tasks_batch_path}task/"
folder_tasks_batch_config_path = f"{folder_tasks_batch_path}config/"

s3_client = boto3.client('s3')

def upload(path, bucket, key, title, content_type, acl=None):
    panel = Panel(f"Region: [italic white on black]{aws_region}[/italic white on black]\nBucket: [italic white on black]{bucket}[/italic white on black]\nFile: [italic white on black]{path}[/italic white on black]\nKey: [italic white on black]{key}[/italic white on black]\nPath: [italic white on black] s3://{aws_region}/{bucket}/{key}[/italic white on black]\nACL: {acl}", title=title)
    console.print(panel)
    if acl:
        response = s3_client.put_object(Body=open(path, 'rb'), Bucket=bucket, Key=key, ContentType=content_type, ACL=acl)
    else:
        response = s3_client.put_object(Body=open(path, 'rb'), Bucket=bucket, Key=key, ContentType=content_type)
    console.print(f"HTTP Status Code: {response['ResponseMetadata']['HTTPStatusCode']}, ETag: {response['ETag']}")

console.rule(f"Task [cyan underline]{task_name}[/cyan underline]/[yellow underline]{batch_name}[/yellow underline] deploy")

console.print(f"[italic purple]deploy-config[/italic purple] variable: {bool(deploy_config)}")

console.print(f"[white on blue bold]Generator configuration")

path = f"{folder_tasks_batch_config_path}admin.json"
key = f"{s3_private_generator_path}admin.json"
upload(path, aws_private_bucket, key, "Admin Credentials", "application/json")

if bool(deploy_config):

    console.print(f"[white on green bold]Task configuration")

    path = f"{folder_tasks_batch_task_path}hits.json"
    key = f"{s3_private_task_path}hits.json"
    upload(path, aws_private_bucket, key, "Hits", "application/json")

    path = f"{folder_tasks_batch_task_path}instructions_dimensions.json"
    key = f"{s3_private_task_path}instructions_dimensions.json"
    upload(path, aws_private_bucket, key, "Assessment Instructions", "application/json")

    path = f"{folder_tasks_batch_task_path}instructions_main.json"
    key = f"{s3_private_task_path}instructions_main.json"
    upload(path, aws_private_bucket, key, "General Instructions", "application/json")

    path = f"{folder_tasks_batch_task_path}questionnaires.json"
    key = f"{s3_private_task_path}questionnaires.json"
    upload(path, aws_private_bucket, key, "Questionnaires", "application/json")

    path = f"{folder_tasks_batch_task_path}dimensions.json"
    key = f"{s3_private_task_path}dimensions.json"
    upload(path, aws_private_bucket, key, "Dimensions", "application/json")

    path = f"{folder_tasks_batch_task_path}search_engine.json"
    key = f"{s3_private_task_path}search_engine.json"
    upload(path, aws_private_bucket, key, "Search Engine", "application/json")

    path = f"{folder_tasks_batch_task_path}task.json"
    key = f"{s3_private_task_path}task.json"
    upload(path, aws_private_bucket, key, "Task Settings", "application/json")

    path = f"{folder_tasks_batch_task_path}workers.json"
    key = f"{s3_private_task_path}workers.json"
    upload(path, aws_private_bucket, key, "Workers Settings", "application/json")

console.print(f"[white on purple bold]Angular Application")

path = f"{folder_tasks_batch_deploy_path}scripts.js"
key = f"{s3_deploy_path}scripts.js"
upload(path, aws_deploy_bucket, key, "Javascript Assets", "text/javascript", "public-read")

path = f"{folder_tasks_batch_deploy_path}styles.css"
key = f"{s3_deploy_path}styles.css"
upload(path, aws_deploy_bucket, key, "CSS Styles", "text/css", "public-read")

path = f"{folder_tasks_batch_deploy_path}index.html"
key = f"{s3_deploy_path}index.html"
upload(path, aws_deploy_bucket, key, "Task Homepage",  "text/html", "public-read")

console.rule("Link")

console.print(f"[bold cyan on black]https://{aws_deploy_bucket}.s3.{aws_region}.amazonaws.com/{task_name}/{batch_name}/index.html")

───────────────────────────────── Task Test/Batch-1 deploy ──────────────────────────────────

deploy-config variable: True

Generator configuration

╭──────────────────────────────────── Admin Credentials ────────────────────────────────────╮
│ Region: us-east-2                                                                         │
│ Bucket: crowdsourcing-tasks-us                                                            │
│ File: tasks/Test/Batch-1/config/admin.json                                                │
│ Key: Test/Batch-1/Generator/admin.json                                                    │
│ Path:  s3://us-east-2/crowdsourcing-tasks-us/Test/Batch-1/Generator/admin.json            │
│ ACL: None                                                                                 │
╰───────────────────────────────────────────────────────────────────────────────────────────╯

HTTP Status Code: 200, ETag: "1c792eb200f297af3eb268b18670012d"

...

HTTP Status Code: 200, ETag: "3ae2c59da3c9b0800b23e8ecca457b72"

╭────────────────────────────────────── Task Homepage ──────────────────────────────────────╮
│ Region: us-east-2                                                                         │
│ Bucket: crowdsourcing-deploy-us                                                           │
│ File: tasks/Test/Batch-1/deploy/index.html                                                │
│ Key: Test/Batch-1/index.html                                                              │
│ Path:  s3://us-east-2/crowdsourcing-deploy-us/Test/Batch-1/index.html                     │
│ ACL: public-read                                                                          │
╰───────────────────────────────────────────────────────────────────────────────────────────╯

HTTP Status Code: 200, ETag: "b726856c6114644edf1f581d18b07e97"

─────────────────────────────────────────── Link ────────────────────────────────────────────

https://crowdsourcing-deploy-us.s3.us-east-2.amazonaws.com/Test/Batch-1/index.html